In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models as models
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split

In [18]:
import ssl
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
ssl._create_default_https_context = ssl._create_unverified_context
data = torchvision.datasets.CIFAR10("./", train=True, download=True, transform=preprocess)
test_data = torchvision.datasets.CIFAR10("./", train=False, transform=preprocess)

Files already downloaded and verified


In [20]:
train_size = int(0.9*len(data))
test_size = len(data) - train_size
train_set, val_set = torch.utils.data.random_split(data, [train_size,test_size])


In [29]:
BATCH_SIZE = 1000
SHUFFLE = True
LEARNING_RATE = 0.001  

train_loader = DataLoader(train_set, batch_size= BATCH_SIZE, shuffle= SHUFFLE )
test_loader = DataLoader(test_data, batch_size= BATCH_SIZE, shuffle= SHUFFLE)
val_loader = DataLoader(val_set, batch_size= BATCH_SIZE, shuffle=False)

In [30]:
model_ft = models.alexnet(pretrained=True)

# Do the things required for fine-tuning before training the model
model_ft.last_linear = nn.Sequential(
    nn.Linear(in_features=100, out_features=100),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=100, out_features=10),
)

In [32]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    best_accuracy = 0
    for epochs in range(num_epochs):
        for batch_nr, (data, labels) in enumerate(train_loader):
         
            prediction = model.forward(data)

            loss = criterion(prediction, labels)

            loss.backward()

            optimizer.step()

            optimizer.zero_grad()

            print(
            f'\rEpoch {epochs+1} [{batch_nr+1}/{len(train_loader)}] - Loss: {loss}',
            end=''
        )
        val_accuracy = 0
        total = 0
        with torch.no_grad():
            for batch_nr, (data, labels) in enumerate(val_loader):
                prediction = model.forward(data)
                _, predicted = torch.max(prediction, 1)
                val_accuracy += (predicted == labels).sum().item()
                total += labels.size(0)
                print(val_accuracy/total, "Val Acc")
            val_accuracy = val_accuracy/total
            if(best_accuracy < val_accuracy):
                best_accuracy = val_accuracy
                print("best model here")
                torch.save(model.state_dict(), "./E2")

def test_model(model, test_loader):
    test_accuracy = 0
    total = 0
    predictList = []
    testList = []
    
    for batch_nr, (data, labels) in enumerate(test_loader):
        prediction = model.forward(data)
        _, predicted = torch.max(prediction, 1)
        for i in range(len(prediction)):
            predictList.append(predicted[i].item())
            testList.append(labels[i].item())
        test_accuracy += (predicted == labels).sum().item()
        total += labels.size(0)


In [33]:
epochs = 2

criterion_ft = nn.CrossEntropyLoss()

# Define our optimizer

optimizer_ft = optim.Adam(model_ft.parameters(), LEARNING_RATE, weight_decay=1e-2)

# Train the model
trained_model_ft = train_model(model_ft, criterion_ft, optimizer_ft, train_loader, val_loader, epochs)
model_ft.load_state_dict(torch.load("./E2"))
# Test the model
tested_model = test_model(model_ft, test_loader)


Epoch 1 [3/45] - Loss: 3.3776037693023689

KeyboardInterrupt: 

In [ ]:
# We use resnet18 as the model.
model_fe = models.alexnet(pretrained=True)

for params in model_fe.parameters():
    params.requires_grad_ = False
# Do the things required for fine-tuning before training the model
model_ft.last_linear = nn.Sequential(
    nn.Linear(in_features=100, out_features=100),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=100, out_features=10),
)

criterion_fe = nn.CrossEntropyLoss()

# Define our optimizer

optimizer_fe = optim.Adam(model_fe.parameters(), LEARNING_RATE, weight_decay=1e-2)

# Train the model

# Train the model
trained_model_fe = train_model(model_fe, criterion_fe, optimizer_fe, train_loader, val_loader, epochs)
model_fe.load_state_dict(torch.load("./E2"))
# Test the model
tested_model = test_model(model_fe, test_loader)